### Simulation Based Inference

#### Exercises Session 3

##### Lorenzo Consoli



##### Exercise 1

Implement rejection sampling to obtain 10000 samples from the posterior distribution for the genetic linkage problem

In [ ]:
import numpy as  np
import matplotlib.pyplot as plt 

from scipy.optimize import minimize_scalar
from tqdm import tqdm

#proposal distribution
def sample_uniform() -> float:
    return np.random.uniform()

def compute_log_likelihood(
        theta: float,
        class_counts: list[int] = [125, 18, 20, 34] 
    ) -> float:
    y1, y2, y3, y4 = class_counts
    likelihood = y1 * np.log(2 + theta) + \
        (y2 + y3) * np.log(1 - theta) + \
            y4 * np.log(theta)
    return - likelihood

def maximize_likelihood() -> float:
    minimized_likelihood = minimize_scalar(compute_log_likelihood, bounds = [0, 1], method = "bounded")
    return - minimized_likelihood.fun

def accept_proposal(
        theta: float,
        unif_sample: float,
        log_C: float
    ) -> bool:
    likelihood = - compute_log_likelihood(theta)
    log_U = np.log(unif_sample)
    #print(f"theta {theta} log_U {log_U} likelihood {likelihood} log_C {log_C}")
    return log_U <= likelihood - log_C

def rejection_sampling() -> float:
    theta = sample_uniform()
    unif_sample = sample_uniform()
    log_C = maximize_likelihood()
    while not accept_proposal(theta, unif_sample, log_C):
        theta = sample_uniform()
    return theta

def genetic_linkage_rejection_sampling(
        n_samples: int = 10000
    ) -> list[float]:
    samples = []
    for sample in tqdm(range(n_samples)): 
        theta = rejection_sampling()
        samples.append(theta)
    return samples

In [ ]:
theta = genetic_linkage_rejection_sampling()

Plot a histogram based on these samples

In [ ]:
plt.hist(theta, bins=50)
plt.show()

Approximate the posterior mean and standard deviation using
these samples

In [ ]:
print(f"posterior mean {np.mean(theta)} posterior std {np.std(theta)}")

In [ ]:
type(genetic_linkage_rejection_sampling)

In [ ]:
from scipy.stats import t, norm
import numpy as np

# Define the function to evaluate f(x) = |x|
def f(x):
    return np.abs(x)

# Define the unnormalized density in the logarithmic scale
def log_p_bar(x):
    return -2 * np.log(1 + x**2/3)

# Define the logarithm of the probability density function for proposal distribution 1 (t-distribution with 3 degrees of freedom)
def log_q1(x):
    return t(df=3).logpdf(x)

# Define the logarithm of the probability density function for proposal distribution 2 (t-distribution with 1 degree of freedom)
def log_q2(x):
    return t(df=1).logpdf(x)

# Define the logarithm of the probability density function for proposal distribution 3 (standard Normal distribution)
def log_q3(x):
    return norm(loc=0, scale=1).logpdf(x)

# Define the logarithm of the importance weight function for proposal distribution 1
def log_w1(x):
    return log_p_bar(x) - log_q1(x)

# Define the logarithm of the importance weight function for proposal distribution 2
def log_w2(x):
    return log_p_bar(x) - log_q2(x)

# Define the logarithm of the importance weight function for proposal distribution 3
def log_w3(x):
    return log_p_bar(x) - log_q3(x)

In [ ]:
# Define x values to evaluate the density functions
x = np.linspace(-5, 5, 1000)

# Evaluate the density functions for each proposal distribution
p1 = t(df=3).pdf(x)
p2 = t(df=1).pdf(x)
p3 = norm(loc=0, scale=1).pdf(x)

# Evaluate the weight functions for each proposal distribution
w1 = np.exp(log_w1(x))
w2 = np.exp(log_w2(x))
w3 = np.exp(log_w3(x))

# Plot the probability density functions of all three proposal distributions
plt.plot(x, p1, label='t-distribution with 3 degrees of freedom')
plt.plot(x, p2, label='t-distribution with 1 degree of freedom')
plt.plot(x, p3, label='standard Normal distribution')
plt.legend()
plt.title('Probability Density Functions of Proposal Distributions')
plt.xlabel('x')
plt.ylabel('p(x)')
plt.show()

# Plot the weight functions of all three proposal distributions
plt.plot(x, w1, label='t-distribution with 3 degrees of freedom')
plt.plot(x, w2, label='t-distribution with 1 degree of freedom')
plt.plot(x, w3, label='standard Normal distribution')
plt.legend()
plt.title('Weight Functions of Proposal Distributions')
plt.xlabel('x')
plt.ylabel('w(x)')
plt.show()

In [ ]:
# Define the function to be integrated
f = lambda x: np.abs(x)
log_p_bar = lambda x: -2 * np.log(1 + x**2/3)

def importance_sampling(q, log_w, N):
    x = q.rvs(N)
    w = np.exp(log_w(x))
    w_norm = w / np.sum(w)
    I = np.sum(w_norm * f(x))
    log_Z = np.log(np.sum(w)) - np.log(N)
    ess = (np.sum(w_norm)*2) / np.sum(w_norm*2)
    return I, log_Z, ess

# Proposal distribution 1: t-distribution with 3 degrees of freedom
q1 = t(df=3)
log_q1 = lambda x: t.logpdf(x, df=3)
log_w1 = lambda x: log_p_bar(x) - log_q1(x)
I1, log_Z1, ess1 = importance_sampling(q1, log_w1, N=10000)

# Proposal distribution 2: t-distribution with 1 degree of freedom
q2 = t(df=1)
log_q2 = lambda x: t.logpdf(x, df=1)
log_w2 = lambda x: log_p_bar(x) - log_q2(x)
I2, log_Z2, ess2 = importance_sampling(q2, log_w2, N=10000)

# Proposal distribution 3: standard Normal distribution
q3 = norm()
log_q3 = lambda x: norm.logpdf(x)
log_w3 = lambda x: log_p_bar(x) - log_q3(x)
I3, log_Z3, ess3 = importance_sampling(q3, log_w3, N=10000)

print("Proposal distribution 1: t-distribution with 3 degrees of freedom")
print("I = ", I1)
print("log Z = ", log_Z1)
print("Effective sample size = ", ess1)
print()
print("Proposal distribution 2: t-distribution with 1 degree of freedom")
print("I = ", I2)
print("log Z = ", log_Z2)
print("Effective sample size = ", ess2)
print()
print("Proposal distribution 3: standard Normal distribution")
print("I = ", I3)
print("log Z = ", log_Z3)
print("Effective sample size = ", ess3)

The effective sample size (ESS) refers to the number of independent samples from the importance sampling distribution that are equivalent to the sample's size. A higher ESS value indicates a more efficient estimator since it reduces the variance of the estimator.

Based on your research, the ESS for the first proposal distribution (a t-distribution with three degrees of freedom) is the highest possible value of 10,000, making it the most effective. The ESS for the second proposal distribution (a t-distribution with one degree of freedom) is lower at 8,642.80, while the ESS for the third proposal distribution (a typical normal distribution) is even lower at 8,132.26.

Therefore, it can be concluded that the first proposal distribution is the most effective, while the third proposal distribution is the least effective. The second proposal distribution falls in the middle. The reason for the difference in ESS values is that the first proposal distribution is closer to the target distribution, resulting in more suggested samples with higher weights, leading to a more effective estimator.